# Exp00918 n=64 Validation Run

In [1]:
library(dplyr)
library(Seurat)
library(patchwork)
library(H5weaver)
library(hise)
library(tidyverse)
library(SeuratObject)
library(ggrepel)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: Matrix

Loading required package: rhdf5


Attaching package: ‘H5weaver’


The following objects are masked from ‘package:rhdf5’:

    h5dump, h5ls


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.4.3     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ 

In [2]:
h5s_w1 <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/scrublet_fig4/w1',
    pattern = 'filtered_feature_bc_matrix.h5$', 
    full.names = TRUE, 
    recursive = TRUE
)

scrub_w1 <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/scrublet_fig4/w1',
    pattern = '.csv$', 
    full.names = TRUE, 
    recursive = TRUE
) %>% 
    lapply(read.csv) %>% 
    lapply(function(x){
        rownames(x) <- x$X
        x$X <- NULL
        x$Barcodes <- rownames(x)
        return(x)
    })


In [3]:
h5s_w2 <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/scrublet_fig4/w2',
    pattern = 'filtered_feature_bc_matrix.h5$',
    full.names = TRUE, 
    recursive = TRUE
)

scrub_w2 <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/scrublet_fig4/w2',
    pattern = '.csv$',
    full.names = TRUE, 
    recursive = TRUE
) %>% 
    lapply(read.csv) %>% 
    lapply(function(x){
        rownames(x) <- x$X
        x$X <- NULL
        x$Barcodes <- rownames(x)
        return(x)
    })


In [4]:
h5s_w3 <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/scrublet_fig4/w3',
    pattern = 'filtered_feature_bc_matrix.h5$', 
    full.names = TRUE, 
    recursive = TRUE
)

scrub_w3 <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/scrublet_fig4/w3',
    pattern = '.csv$',
    full.names = TRUE, 
    recursive = TRUE
) %>% 
    lapply(read.csv) %>% 
    lapply(function(x){
        rownames(x) <- x$X
        x$X <- NULL
        x$Barcodes <- rownames(x)
        return(x)
    })


In [5]:
h5s_w4 <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/scrublet_fig4/w4',
    pattern = 'filtered_feature_bc_matrix.h5$', 
    full.names = TRUE, 
    recursive = TRUE
)

scrub_w4 <- list.files(
    path = '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/scrublet_fig4/w4',
    pattern = '.csv$', 
    full.names = TRUE, 
    recursive = TRUE
) %>% 
    lapply(read.csv) %>% 
    lapply(function(x){
        rownames(x) <- x$X
        x$X <- NULL
        x$Barcodes <- rownames(x)
        return(x)
    })


In [6]:
so_gen <- function(x){
    # pro <- strsplit(strsplit(x,'/per_sample_outs/')[[1]][2],'/count/')[[1]][1]
    pro <- strsplit(x, '/')[[1]][10] 
    pro <- strsplit(pro, '_')[[1]][1]
    
    exp <- strsplit(x, '/')[[1]][9] 

    pro <- paste(pro,exp,sep='_')

    mtx <- Read10X_h5(x) 
    so <- CreateSeuratObject(mtx,project=pro) 
    return(so) 
    }

In [7]:
fixed_so_w1 <- lapply(h5s_w1, so_gen)
fixed_so_w2 <- lapply(h5s_w2, so_gen)
fixed_so_w3 <- lapply(h5s_w3, so_gen)
fixed_so_w4 <- lapply(h5s_w4, so_gen)


In [8]:
scrub.it <- function(so_l, scrub_l){
    for (i in c(1:length(so_l))){
        so_l[[i]][[]]$Barcodes <- rownames(so_l[[i]][[]])
        # so_l[[i]][[]]$Barcodes <- rownames(scrub_l[[i]])
        so_l[[i]][[]] <- left_join(so_l[[i]][[]], scrub_l[[i]], by = 'Barcodes') # , by='row.names', all=TRUE
    }
    return(so_l)
}

In [9]:
fixed_so_w1 <- fixed_so_w1 %>% scrub.it(scrub_w1)
fixed_so_w2 <- fixed_so_w2 %>% scrub.it(scrub_w2)
fixed_so_w3 <- fixed_so_w3 %>% scrub.it(scrub_w3)
fixed_so_w4 <- fixed_so_w4 %>% scrub.it(scrub_w4)

In [10]:
fully_w1 <- Reduce(merge,fixed_so_w1) %>% JoinLayers()
fully_w2 <- Reduce(merge,fixed_so_w2) %>% JoinLayers()
fully_w3 <- Reduce(merge,fixed_so_w3) %>% JoinLayers()
fully_w4 <- Reduce(merge,fixed_so_w4) %>% JoinLayers()


In [11]:
rm(fixed_so_w1)
rm(fixed_so_w2)
rm(fixed_so_w3)
rm(fixed_so_w4)

In [12]:
fully_w1[["percent.mt"]] <- PercentageFeatureSet(fully_w1, pattern = "^MT-")
fully_w2[["percent.mt"]] <- PercentageFeatureSet(fully_w2, pattern = "^MT-")
fully_w3[["percent.mt"]] <- PercentageFeatureSet(fully_w3, pattern = "^MT-")
fully_w4[["percent.mt"]] <- PercentageFeatureSet(fully_w4, pattern = "^MT-")


In [13]:
fully_w1 <- subset(fully_w1, subset = percent.mt < 5)
fully_w2 <- subset(fully_w2, subset = percent.mt < 5)
fully_w3 <- subset(fully_w3, subset = percent.mt < 5)
fully_w4 <- subset(fully_w4, subset = percent.mt < 5)


In [14]:
fully_w1 <- fully_w1 %>% subset(subset=Predicted_Doublet=='False')
fully_w2 <- fully_w2 %>% subset(subset=Predicted_Doublet=='False')
fully_w3 <- fully_w3 %>% subset(subset=Predicted_Doublet=='False')
fully_w4 <- fully_w4 %>% subset(subset=Predicted_Doublet=='False')


## Plotting

In [15]:
fully_w1$well <- substr(fully_w1$orig.ident, 13,14)
fully_w2$well <- substr(fully_w2$orig.ident, 13,14)
fully_w3$well <- substr(fully_w3$orig.ident, 13,14)
fully_w4$well <- substr(fully_w4$orig.ident, 13,14)

fully_w1$donor <- substr(fully_w1$orig.ident,1,7)
fully_w2$donor <- substr(fully_w2$orig.ident,1,7)
fully_w3$donor <- substr(fully_w3$orig.ident,1,7)
fully_w4$donor <- substr(fully_w4$orig.ident,1,7)

fully_w1$donor2 <- substr(fully_w1$orig.ident,1,11)
fully_w2$donor2 <- substr(fully_w2$orig.ident,1,11)
fully_w3$donor2 <- substr(fully_w3$orig.ident,1,11)
fully_w4$donor2 <- substr(fully_w4$orig.ident,1,11)


In [16]:
head(fully_w1)

,orig.ident,nCount_RNA,nFeature_RNA,Barcodes,Doublet_Score,Predicted_Doublet,percent.mt,well,donor,donor2
,<chr>,<dbl>,<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
AAACAAGCACCGGTAGACTTTAGG-1,BL05013-033_w1,6654,3198,AAACAAGCACCGGTAGACTTTAGG-1,0.04133545,False,2.1791404,w1,BL05013,BL05013-033
AAACAAGCACTCCCTTACTTTAGG-1,BL05013-033_w1,14577,4339,AAACAAGCACTCCCTTACTTTAGG-1,0.01758642,False,0.4047472,w1,BL05013,BL05013-033
AAACAAGCATTCCTTTACTTTAGG-1,BL05013-033_w1,11459,4099,AAACAAGCATTCCTTTACTTTAGG-1,0.01044530,False,0.9424906,w1,BL05013,BL05013-033
AAACAAGCATTTAGCCACTTTAGG-1,BL05013-033_w1,5790,2967,AAACAAGCATTTAGCCACTTTAGG-1,0.02735978,False,2.6943005,w1,BL05013,BL05013-033
AAACCAATCAGTAGCGACTTTAGG-1,BL05013-033_w1,4833,2662,AAACCAATCAGTAGCGACTTTAGG-1,0.05347594,False,1.6139044,w1,BL05013,BL05013-033
AAACCAATCCCTGGTGACTTTAGG-1,BL05013-033_w1,7424,3490,AAACCAATCCCTGGTGACTTTAGG-1,0.01680672,False,0.8890086,w1,BL05013,BL05013-033
AAACCAATCCTGACTAACTTTAGG-1,BL05013-033_w1,7406,3448,AAACCAATCCTGACTAACTTTAGG-1,0.07843137,False,1.5527950,w1,BL05013,BL05013-033
AAACCAATCGACCGTTACTTTAGG-1,BL05013-033_w1,6069,3089,AAACCAATCGACCGTTACTTTAGG-1,0.08371041,False,2.2903279,w1,BL05013,BL05013-033
AAACCAATCGGTTATTACTTTAGG-1,BL05013-033_w1,5214,2816,AAACCAATCGGTTATTACTTTAGG-1,0.06072106,False,1.7453011,w1,BL05013,BL05013-033


In [17]:
fully <- merge(fully_w1, c(fully_w2, fully_w3, fully_w4)) %>% JoinLayers()


Warning message:
“Some cell names are duplicated across objects provided. Renaming to enforce unique cell names.”


In [18]:
head(fully)

,orig.ident,nCount_RNA,nFeature_RNA,Barcodes,Doublet_Score,Predicted_Doublet,percent.mt,well,donor,donor2
,<chr>,<dbl>,<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
AAACAAGCACCGGTAGACTTTAGG-1_1,BL05013-033_w1,6654,3198,AAACAAGCACCGGTAGACTTTAGG-1,0.04133545,False,2.1791404,w1,BL05013,BL05013-033
AAACAAGCACTCCCTTACTTTAGG-1_1,BL05013-033_w1,14577,4339,AAACAAGCACTCCCTTACTTTAGG-1,0.01758642,False,0.4047472,w1,BL05013,BL05013-033
AAACAAGCATTCCTTTACTTTAGG-1_1,BL05013-033_w1,11459,4099,AAACAAGCATTCCTTTACTTTAGG-1,0.01044530,False,0.9424906,w1,BL05013,BL05013-033
AAACAAGCATTTAGCCACTTTAGG-1_1,BL05013-033_w1,5790,2967,AAACAAGCATTTAGCCACTTTAGG-1,0.02735978,False,2.6943005,w1,BL05013,BL05013-033
AAACCAATCAGTAGCGACTTTAGG-1_1,BL05013-033_w1,4833,2662,AAACCAATCAGTAGCGACTTTAGG-1,0.05347594,False,1.6139044,w1,BL05013,BL05013-033
AAACCAATCCCTGGTGACTTTAGG-1_1,BL05013-033_w1,7424,3490,AAACCAATCCCTGGTGACTTTAGG-1,0.01680672,False,0.8890086,w1,BL05013,BL05013-033
AAACCAATCCTGACTAACTTTAGG-1_1,BL05013-033_w1,7406,3448,AAACCAATCCTGACTAACTTTAGG-1,0.07843137,False,1.5527950,w1,BL05013,BL05013-033
AAACCAATCGACCGTTACTTTAGG-1_1,BL05013-033_w1,6069,3089,AAACCAATCGACCGTTACTTTAGG-1,0.08371041,False,2.2903279,w1,BL05013,BL05013-033
AAACCAATCGGTTATTACTTTAGG-1_1,BL05013-033_w1,5214,2816,AAACCAATCGGTTATTACTTTAGG-1,0.06072106,False,1.7453011,w1,BL05013,BL05013-033


In [19]:
fully <- NormalizeData(fully) %>% 
    FindVariableFeatures() %>% 
    ScaleData() %>% 
    RunPCA() %>% 
    RunUMAP(dims = 1:20) %>% 
    FindNeighbors(dims = 1:20) %>% 
    FindClusters(resolution = 0.5)

Normalizing layer: counts

Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  CD247, TRBC1, LEF1, CCR7, CD69, TRAT1, MAL, CD27, NELL2, MYC 
	   PYHIN1, SAMD3, OBSCN, CTSW, HIST1H1D, HIST1H1C, AQP3, IL2RB, CENPM, KLRK1 
	   GATA3, KLRG1, GBP5, CD40LG, SH2D2A, ICOS, ARID5B, DPP4, PRDM1, MYBL1 
Negative:  CLEC7A, SERPINA1, TNFAIP2, PLXNB2, EMILIN2, NCF2, SPI1, CST3, LILRB2, HCK 
	   IFI30, LRRC25, MNDA, MEFV, HK3, LYZ, TYMP, CLEC12A, NFAM1, CD68 
	   FGL2, CD300E, SLC11A1, PILRA, PRAM1, LRP1, S100A9, TGFBI, ZNF385A, CSF3R 
PC_ 2 
Positive:  CD79A, MS4A1, PAX5, NIBAN3, BANK1, IGHM, CD22, FCRL1, IGHD, FCRL2 
	   TNFRSF13C, BLK, POU2AF1, BCL11A, RALGPS2, RUBCNL, IGKC, CD79B, WDFY4, FCRLA 
	   CD19, FCER2, OSBPL10, SWAP70, AFF3, COBLL1, ADAM28, BLNK, CIITA, CDK14 
Negative:  SAMHD1, SPN, ANXA1, CD247, SRGN, ITGB2, ADGRE5, CTSW, PRF1, SAMD3 
	   ID2, S100A4, NKG7, CST7, IL2RB, GZMA, MYO1F, SYNE1, CD300A, DOK2 
	   FOSL2, CCL5, KLRD1, GNLY, KLRK1, M

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 411871
Number of edges: 10507337

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9410
Number of communities: 31
Elapsed time: 904 seconds


1 singletons identified. 30 final clusters.



In [20]:
saveRDS(fully, '/home/jupyter/CS15_WHBL/CWB_Paper/01_Final_Data/04_Data/Fig_4_Final.rds')

In [22]:
head(fully)

,orig.ident,nCount_RNA,nFeature_RNA,Barcodes,Doublet_Score,Predicted_Doublet,percent.mt,well,donor,donor2,RNA_snn_res.0.5,seurat_clusters
,<chr>,<dbl>,<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<fct>,<fct>
AAACAAGCACCGGTAGACTTTAGG-1_1,BL05013-033_w1,6654,3198,AAACAAGCACCGGTAGACTTTAGG-1,0.04133545,False,2.1791404,w1,BL05013,BL05013-033,3,3
AAACAAGCACTCCCTTACTTTAGG-1_1,BL05013-033_w1,14577,4339,AAACAAGCACTCCCTTACTTTAGG-1,0.01758642,False,0.4047472,w1,BL05013,BL05013-033,2,2
AAACAAGCATTCCTTTACTTTAGG-1_1,BL05013-033_w1,11459,4099,AAACAAGCATTCCTTTACTTTAGG-1,0.01044530,False,0.9424906,w1,BL05013,BL05013-033,10,10
AAACAAGCATTTAGCCACTTTAGG-1_1,BL05013-033_w1,5790,2967,AAACAAGCATTTAGCCACTTTAGG-1,0.02735978,False,2.6943005,w1,BL05013,BL05013-033,5,5
AAACCAATCAGTAGCGACTTTAGG-1_1,BL05013-033_w1,4833,2662,AAACCAATCAGTAGCGACTTTAGG-1,0.05347594,False,1.6139044,w1,BL05013,BL05013-033,0,0
AAACCAATCCCTGGTGACTTTAGG-1_1,BL05013-033_w1,7424,3490,AAACCAATCCCTGGTGACTTTAGG-1,0.01680672,False,0.8890086,w1,BL05013,BL05013-033,9,9
AAACCAATCCTGACTAACTTTAGG-1_1,BL05013-033_w1,7406,3448,AAACCAATCCTGACTAACTTTAGG-1,0.07843137,False,1.5527950,w1,BL05013,BL05013-033,3,3
AAACCAATCGACCGTTACTTTAGG-1_1,BL05013-033_w1,6069,3089,AAACCAATCGACCGTTACTTTAGG-1,0.08371041,False,2.2903279,w1,BL05013,BL05013-033,1,1
AAACCAATCGGTTATTACTTTAGG-1_1,BL05013-033_w1,5214,2816,AAACCAATCGGTTATTACTTTAGG-1,0.06072106,False,1.7453011,w1,BL05013,BL05013-033,1,1


In [21]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.4 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] ggrepel_0.9.5      lubridate_1.9.3    forcats_1.0.0      stringr_1.5.1     
 [5] purrr_1.0.2        readr_2.1.5        tidyr_1.3.1        tibble_3.2.1      
 [9] ggplot2_3.4.3      tidyverse_2.0.0    hise_2.15.0        H5weaver_1.2.0    
[13] rhdf5_2.38.1       Matrix_1.6-4       data.table_1.14.2  patchwo